# Carregando variáveis de ambiente

In [82]:
from dotenv import dotenv_values

config = dotenv_values(".env")

pinecone_api_key = config["PINECONE_API_KEY"]
pinecone_env = config["PINECONE_ENVIRONMENT"]
index_pinecone = config["INDEX_NAME"]

openai_api_key = config["OPENAI_API_KEY"]

# Verificando conexão com o pinecone

In [83]:
from pinecone import Pinecone

pc = Pinecone(api_key=pinecone_api_key)
index = pc.Index(index_pinecone)

print(pc.list_indexes())

[{
    "name": "ods-onu",
    "metric": "cosine",
    "host": "ods-onu-y4phmo4.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": {
        "embedding_model": "text-embedding-3-small"
    }
}, {
    "name": "livreto-base-evidencia",
    "metric": "cosine",
    "host": "livreto-base-evidencia-y4phmo4.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": {
        "embedding_model": "text-embedding-3-small"
    }
}]


# Extraindo texto do pdf (ods_onu.pd) e salvando em formato json (ods_onu.json)

In [84]:
import pdfplumber
import json

path = "./data/ods_onu.pdf"
pages=[]
with pdfplumber.open(path) as pdf:
    for i , p in enumerate(pdf.pages, start=1):
        if i < 18 or i > 39:
            continue
        text = p.extract_text() or ""
        pages.append({"page": i, "text": text})

with open("json/ods_onu.json","w",encoding="utf-8") as f:
    json.dump(pages,f,ensure_ascii=False,indent=2)

# Limpando textos extraídos no json e salvando novo json limpo

In [85]:
import re 

def limpar_texto_ods(texto: str) -> str:
    if not texto:
        return ""

    # Mantemos sua lógica, mas garantimos que pontos entre números (1.1) não sumam
    texto = re.sub(r'[^\wÀ-ÿ\s.,;:!?()\-"“”]', ' ', texto)
    
    # Remove letras soltas (OCR)
    texto = re.sub(r'(?:\b[A-ZÀ-Ý]\b\s*){3,}', ' ', texto)

    # Normalização de quebras e espaços
    texto = re.sub(r'\n{2,}', '\n', texto)
    texto = re.sub(r'\n', ' ', texto)
    texto = re.sub(r'\s{2,}', ' ', texto)

    return texto.strip()


def limpar_json_ods(lista_ods: list) -> list:
    json_limpo = []
    for pagina in lista_ods:
        # Aqui, 'text' deve conter a descrição da ODS e suas Metas
        texto_limpo = limpar_texto_ods(pagina.get("text", ""))
        
        if texto_limpo:
            json_limpo.append({
                "page": pagina["page"],
                "text": texto_limpo
            })
    return json_limpo



with open('json/ods_onu.json', 'r', encoding='utf-8') as f:
    raw_data = json.load(f)


ods_limpas = limpar_json_ods(raw_data)

with open("json/ods_onu_clean.json", "w", encoding="utf-8") as f:
    json.dump(ods_limpas, f, ensure_ascii=False, indent=2)

# Fazendo chunks

In [86]:
from langchain_core.documents import Document


with open('json/ods_onu_clean.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

def preparar_documentos_ods(data):
    docs_com_contexto = []
    
    for item in data:
        texto = item['text']
        # Divide o texto em sentenças ou metas baseando-se no padrão "Objetivo X" ou "X.X"
        # Aqui quebramos onde começa um novo Objetivo ou uma nova Meta
        partes = re.split(r'(Objetivo \d+\.)', texto)
        
        for i in range(1, len(partes), 2):
            titulo_ods = partes[i].strip()  # Ex: "Objetivo 1."
            conteudo_ods = partes[i+1].strip() # O texto que vem depois
            
            # Se for o detalhamento de metas (ex: 1.1, 1.2)
            # Vamos garantir que o título da ODS acompanhe cada bloco
            doc = Document(
                page_content=f"{titulo_ods} {conteudo_ods}",
                metadata={
                    "page": item['page'],
                    "ods": titulo_ods
                }
            )
            docs_com_contexto.append(doc)
            
    return docs_com_contexto

documents = preparar_documentos_ods(data)
documents

[Document(metadata={'page': 18, 'ods': 'Objetivo 1.'}, page_content='Objetivo 1. Acabar com a pobreza em todas as suas formas, em todos os lugares'),
 Document(metadata={'page': 18, 'ods': 'Objetivo 2.'}, page_content='Objetivo 2. Acabar com a fome, alcançar a segurança alimentar e melhoria da nutrição e promover a agricultura sustentável'),
 Document(metadata={'page': 18, 'ods': 'Objetivo 3.'}, page_content='Objetivo 3. Assegurar uma vida saudável e promover o bem-estar para todos, em todas as idades'),
 Document(metadata={'page': 18, 'ods': 'Objetivo 4.'}, page_content='Objetivo 4. Assegurar a educação inclusiva e equitativa e de qualidade, e promover oportunidades de aprendizagem ao longo da vida para todos'),
 Document(metadata={'page': 18, 'ods': 'Objetivo 5.'}, page_content='Objetivo 5. Alcançar a igualdade de gênero e empoderar todas as mulheres e meninas'),
 Document(metadata={'page': 18, 'ods': 'Objetivo 6.'}, page_content='Objetivo 6. Assegurar a disponibilidade e gestão sust

# Fazendo embeddings

In [87]:
from langchain_text_splitters import RecursiveCharacterTextSplitter 

# 1. Definimos os separadores específicos para a estrutura das ODS
# A ordem importa: ele tenta o primeiro, se não conseguir (chunk muito grande), tenta o segundo...
separadores_ods = [
    "Objetivo ",      # Garante que cada ODS comece em um novo chunk se possível
    "\n",             # Quebra de linha
    r"(?=\d+\.\d+)",   # Regex: Quebra ANTES de metas numéricas como 1.1, 2.3
    r"(?=\d+\.[a-z])", # Regex: Quebra ANTES de metas com letras como 1.a, 2.b
    ". ",             # Pontos finais
    " "               # Espaços
]

# 2. Configuramos o Splitter
text_splitter_ods = RecursiveCharacterTextSplitter(
    chunk_size=1200,   # Tamanho ideal para conter uma ODS e algumas de suas metas
    chunk_overlap=150, # Sobreposição para garantir que o contexto não se perca na virada de chunk
    separators=separadores_ods,
    is_separator_regex=True # Habilita o uso de Regex para identificar as metas (1.1, 4.a)
)


# 3. Aplicamos aos seus documentos de ODS
# Supondo que sua lista de Documents se chame 'documents_ods'
chunks_ods = text_splitter_ods.split_documents(documents)

# Verificando a quantidade de chunks gerados
print(f"Total de documentos originais: {len(documents)}")
print(f"Total de chunks gerados: {len(chunks_ods)}")

Total de documentos originais: 34
Total de chunks gerados: 49


In [88]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore


# Inicializando o modelo de embeddings da OpenAI
# O modelo 'text-embedding-3-small' é o mais custo-benefício atualmente
embeddings_model = OpenAIEmbeddings(
    api_key=openai_api_key, 
    model="text-embedding-3-small"
)


# Enviando para um "espaço" chamado 'catalogo-ods'
vector_store_ods = PineconeVectorStore.from_documents(
    documents=chunks_ods,
    embedding=embeddings_model,
    index_name=index_pinecone, # O mesmo índice que você já usa
    pinecone_api_key=pinecone_api_key,
    namespace="catalogo-ods"    # A separação mágica acontece aqui
)

print("Upload das ODS concluído com sucesso no namespace 'catalogo-ods'!")

Upload das ODS concluído com sucesso no namespace 'catalogo-ods'!


# Chamando o outro vector store 'livreto-base-evidencia'

In [89]:
# Esse vector já foi criado em outro projeto, por isso só precisei instacia-lo aqui 
vector_store_livreto = PineconeVectorStore(
    index_name="livreto-base-evidencia",
    embedding=embeddings_model,
    pinecone_api_key=pinecone_api_key,
)

# Definindo retrievals

In [90]:
retriever_livreto = vector_store_livreto.as_retriever(
    search_kwargs={
        "namespace": "livreto-metricis", 
        "k": 3
        }
    ) 


retriever_ods = vector_store_ods.as_retriever(
    search_kwargs={
        "namespace": "catalogo-ods",
        "k": 2
        }
    )

# Definindo tools para o agente

In [ ]:
from langchain.tools.retriever import create_retriever_tool

# Aqui, você transforma os retrievers e uma função Python em ferramentas que o Agente sabe usar:
tool_livreto = create_retriever_tool(
    retriever_livreto,
    "busca_gestao_publica",
    "Útil para identificar qual etapa do ciclo de políticas públicas se relaciona com um texto ou projeto."
)

tool_ods = create_retriever_tool(
    retriever_ods,
    "busca_ods_onu",
    "Útil para identificar quais Objetivos de Desenvolvimento Sustentável (ODS) e metas da ONU se relacionam com um texto ou projeto."
)

# tool_livreto e tool_ods: Permitem que o Agente "saia para pesquisar" se não tiver certeza sobre uma ODS ou uma etapa do ciclo de políticas.


In [ ]:
from enum import Enum
from pydantic import Field, BaseModel
from typing import List

class EtapaCicloPP(Enum):
    DEFINICAO = "Definição e Dimensão"
    MOBILIZACAO = "Mobilização"
    MAPEAMENTO = "Mapeamento dos Determinantes"
    SOLUCAO = "Solução"
    JUSTIFICATIVA = "Justificativa"
    APRIMORAMENTO = "Aprimoramento"
    CERTIFICACAO = "Certificação"

class Classificacao(Enum):
    ACADEMICA = "Acadêmica"
    TECNICA = "Técnica"

class AreaAvaliada(Enum):
    EDUCACAO = "Educação"
    SAUDE = "Saúde"
    MEIO_AMBIENTE = "Meio Ambiente"
    GENERO = "Gênero"
    RACA = "Raça"
    POBREZA = "Pobreza"
    DESENVOLVIMENTO_SOCIAL = "Desenvolvimento Social"


class Metodologia(Enum):
    QUALITATIVA = "Qualitativa"
    QUANTITATIVA = "Quantitativa"
    MISTA = "Mista"


class Trabalho(BaseModel):
    classificacao: Classificacao = Field(..., description="Acadêmico ou técnico")
    metodologia: Metodologia = Field(..., description="Abordagem do trabalho")
    area_avaliada: AreaAvaliada = Field(..., description="Área avaliada no trabalho")
    ods: List[int] = Field(..., description="Número das ODS relacionadas ao trabalho (texto)")
    etapa: EtapaCicloPP = Field(..., description="Etapa do ciclo de políticas públicas")
    titulo:str = Field(..., description="Titulo do trabalho")



from langchain.tools import tool

# 1. Criamos uma função decorada como @tool
# Isso transforma seu Pydantic em uma ferramenta que o Agente reconhece
@tool
def formatar_resposta_final(analise: Trabalho):
    """
    Usa esta ferramenta por último para entregar a análise final estruturada. 
    Esta ferramenta valida todos os Enums e campos obrigatórios.
    """
    return analise
tools_com_output = [tool_livreto, tool_ods, formatar_resposta_final]

# Criando agente

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


def agente_classificador(query:str) -> dict:
    # 1. O system_text: É a "Constituição" do Agente. Aqui você define o Codebook.
    #  Você deu à IA a "personalidade" de um pesquisador sênior e explicou cada regra de negócio do Insper. 
    system_text = """
        Você é um pesquisador sênior especializado em revisão sistemática de políticas públicas e 
        nos Objetivos de Desenvolvimento Sustentável (ODS) da ONU. 
        Sua tarefa é codificar estudos extraindo dados sistematicamente conforme os critérios abaixo.

        ### CRITÉRIOS DE CODIFICAÇÃO (CODEBOOK)

        1. CLASSIFICAÇÃO:
        - "Academica": Foco em rigor metodológico, fundamentação teórica e revisão de literatura.
        - "Tecnica": Foco em execução, procedimentos operacionais e resultados práticos/gestão.

        2. METODOLOGIA:
        - "Qualitativa", "Quantitativa" ou "Mista".

        3. ÁREA AVALIADA:
        - Escolha uma: "Educação", "Saúde", "Meio Ambiente", "Gênero", "Raça", "Pobreza" ou "Desenvolvimento Social".

        4. ETAPAS DO CICLO DE POLÍTICAS PÚBLICAS (Base: Insper/IAS 2019):
        - "Definição e Dimensão": Especificação, mensuração e consequências do problema.
        - "Mobilização": Sensibilização, percepção de atores-chave e eficácia da mobilização.
        - "Mapeamento dos Determinantes": Identificação e priorização de causas modificáveis.
        - "Solução": Estratégia, modelo de mudança, validade das hipóteses e metas.
        - "Justificativa": Descrição de impacto, valoração de custos/benefícios e custo-efetividade.
        - "Aprimoramento": Monitoramento, eficiência, eficácia alocativa e validação de implementação.
        - "Certificação": Estimativas finais de impacto, custo final, adequação e resolutividade.

        5. ODS (Objetivos de Desenvolvimento Sustentável):
        - Identifique as ODS (1 a 17) mais relacionadas à temática central.
        - Exemplo de formato: "ODS 1; ODS 5; ODS 10".

        ### TAREFA
        Sempre que receber um texto, analise-o profundamente contra os critérios acima e 
        responda **APENAS** com um objeto JSON válido. Não inclua textos explicativos fora do JSON.

        ### FORMATO DE SAÍDA (OBRIGATÓRIO)
        {{
            "titulo": "Título original do trabalho",
            "classificacao": "Academica ou Tecnica",
            "metodologia": "Qualitativa, Quantitativa ou Mista",
            "area_avaliada": "Área correspondente",
            "etapa": "Nome da etapa conforme o Codebook",
            "ods": "Lista de ODS (Ex: 3; 4)",
            "resumo_justificativa": "Breve frase justificando a etapa do ciclo escolhida"
        }}
    """
    # Note o uso de chaves duplas {{ }} no formato de saída; 
    # isso é necessário porque o Python entende chaves únicas como variáveis, 
    # então "escapamos" elas para que o Agente entenda que é um texto de estrutura JSON.


    # 2. Definimos o modelo (GPT-4o é o melhor para raciocínio técnico)
    llm = ChatOpenAI(model="gpt-4o", temperature=0, api_key=openai_api_key)




    # 3. Criamos o "Manual de Instruções" do Agente (Prompt)
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_text), # Injeta as regras de pesquisador sênior.
        ("human", "{input}"), # É onde o texto do seu trabalho entrará.
        MessagesPlaceholder(variable_name="agent_scratchpad"), # Esta é a linha mais "mágica". 
        # O scratchpad (bloco de notas) é um espaço dinâmico onde 
        # o LangChain armazena o histórico de quais ferramentas o agente já usou e 
        # o que ele descobriu nelas antes de te dar a resposta final.
    ])



    # 4. Construímos de fato o Agent
    agent = create_openai_functions_agent(llm, tools_com_output, prompt)
    # Esta linha "cola" as três partes: 
    # - o cérebro (llm) 
    # - as mãos (tools_com_output que definimos antes, como o retriever e a formatação) 
    # - as instruções (prompt). Ela cria o raciocínio lógico que sabe quando chamar uma função externa.

    # 5. Criamos o Executor: O Agente em si é apenas uma lógica; o Executor é quem realmente roda o loop "Pensar -> Agir -> Observar".
    agent_executor = AgentExecutor(
        agent=agent, 
        tools=tools_com_output, 
        verbose=True, # com verbose=True para você ver ele 'pensando'
        return_intermediate_steps=False # Queremos apenas o resultado final
    )

    # 6. Execução: O dicionário {"input": query} preenche aquele campo {input} que definimos no prompt lá atrás.
    resposta = agent_executor.invoke({"input": query}) 

    return resposta["output"]

In [94]:
query = """
620 | © RAE | São Paulo | V. 57 | n. 6 | nov-dez 2017 | 620-625 ISSN 0034-7590
RAE-Revista de Administração de Empresas | FGV EAESP
SÉRGIO LAZZARINI
sergiogl1@insper.edu.br
Professor do Insper, Cátedra Chafi
Haddad de Administração – São
Paulo – SP, Brasil
PERSPECTIVAS
Artigo convidado
PESQUISA EM ADMINISTRAÇÃO: EM BUSCA
DE IMPACTO SOCIAL E OUTROS IMPACTOS
Eu não estou interessado
em nenhuma teoria
em nenhuma fantasia
nem no algo mais...
Amar e mudar as coisas
me interessa mais…
Belchior
INTRODUÇÃO
No Encontro da Associação Nacional de Pós-Graduação e Pesquisa em Administração (ANPAD) de
2015, tive o prazer de participar de um painel organizado pelo meu colega Charles Kirschbaum,
centrado no tema “Como aumentar o impacto da pesquisa em Administração”. Em um auditório
lotado, Charles iniciou os debates com um teste de múltipla escolha, mais ou menos nesta linha:
Se um jornalista quisesse saber algo sobre a desigualdade de salários nas organizações,
para quem ligaria?
a. um economista
b. um cientista político
c. um sociólogo
d. um pesquisador em administração
A plateia, na sua maioria composta por pesquisadores em Administração, acabou
maciçamente escolhendo a alternativa A. Para os presentes, o jornalista, muito provavelmente,
ligaria para um economista. Certamente, a explicação para isso não é porque só os economistas
fazem pesquisa sobre salários e sua distribuição nas organizações. Os determinantes de mecanismos
de compensação em empresas sempre foram um tema muito comum no campo da Administração,
há um bom tempo. Por exemplo, Finkelstein e Hambrick (1989) identificaram processos políticos
influenciando a compensação de CEOs; Zenger (1992) usou teorias de comparação social para
explicar a distribuição de salários extremos em hierarquias, no topo e na base; e Hoskisson, Castleton
e Withers (2009) argumentaram que os crescentes salários recebidos por executivos de topo são
DOI: http://dx.doi.org/10.1590/S0034-759020170608
ISSN 0034-7590
PERSPECTIVAS | PESQUISA EM ADMINISTRAÇÃO: EM BUSCA DE IMPACTO SOCIAL E OUTROS IMPACTOS
Sérgio Lazzarini
621 | © RAE | São Paulo | V. 57 | n. 6 | nov-dez 2017 | 620-625
decorrentes de uma maior pressão de acionistas e do crescente
risco que esses executivos têm que assumir nas suas empresas.
Não faltam teorias e estudos em Administração que
poderiam ajudar um pesquisador a dar uma longa e inspirada
entrevista sobre desigualdade de salários. Por que, então, a
própria comunidade de Administração considera que não seria a
primeira escolha do jornalista? Neste artigo, sem ter a pretensão
de dar respostas definitivas, discutirei por que a pesquisa em
Administração acaba tendo, muitas vezes, um impacto percebido
aquém do seu potencial e apresentarei algumas sugestões
sobre o que pode ser feito a esse respeito. Muito embora exista
grande espaço para aumentar o impacto social da pesquisa na
área, a partir de estudos que ajudem a entender e a solucionar
questões sociais prementes, a minha discussão será um pouco
mais geral: Como aumentar o impacto da pesquisa para gestores,
formuladores de políticas e demais atores de entorno que possam
se beneficiar dos achados trazidos por estudos teóricos ou
empíricos sobre processos e estratégias organizacionais?
QUAL IMPACTO?
O impacto acadêmico de uma publicação é usualmente avaliado
pelas citações que um determinado artigo ou livro recebe ao longo
dos anos. Em geral, uma pesquisa academicamente relevante
é aquela que ajuda a construir conhecimento em torno de um
determinado tema, abordagem teórica ou objeto empírico. A
geração de conhecimento, no entanto, é, por si só, um processo
de construção social (Latour, 1987); é possível que determinados
grupos de pesquisadores se referenciem ao longo do tempo
como forma de legitimar ou consolidar um determinado campo,
ainda que esteja divorciado da realidade ou de qualquer tipo
de aplicação. Uma pergunta-chave que emerge nesse contexto
é: Será que uma pesquisa prestigiada por pares acadêmicos
também resulta em maior relevância para compreender ou
solucionar questões reais?
Para examinar essa questão, Baldridge, Floyd e Markóczy
(2004) obtiveram uma amostra aleatória de 120 artigos publicados
em revistas acadêmicas de alto impacto em Administração. Em
linha com a discussão anterior, o impacto acadêmico dos artigos
foi medido pelo seu volume de citações futuras. Para avaliar o
impacto dos artigos no mundo empresarial, os autores construíram
um índice de “relevância” avaliado por um painel de experts com
atuação executiva em organizações. O resultado: os autores
encontraram uma correlação estatisticamente significante entre a
relevância do artigo e o seu impacto acadêmico. Aparentemente,
os artigos mais citados foram, em geral, considerados pelos
executivos como de maior potencial para gerar lições relevantes
para o mundo empresarial. Uma possível explicação é que artigos
mais citados tendem a ser aqueles de maior qualidade e com
resultados mais fortes e surpreendentes, com amplo potencial
de contestar visões preestabelecidas ou lançar luz sobre novos
conceitos ou relações entre variáveis de alta relevância.
É claro que esses resultados se baseiam em correlações.
Os autores citam o caso de artigos julgados relevantes pelos
executivos, mas que não receberam uma única citação. Há,
também, artigos muito citados e julgados pouco relevantes –
esses podem ser aqueles ligados a debates teóricos cerceados no
mundo dos pesquisadores e com pouca ancoragem a problemas
reais. Por certo, há debates teóricos que não trazem aplicação
imediata, porém abrem espaço para avanços aplicados relevantes.
Por exemplo, a discussão teórica trazida por Granovetter (1985),
apresentando sua perspectiva de embededdness em oposição
a abordagens com raiz em racionalidade econômica, foi
fundamental para abrir toda uma nova linha de pesquisa aplicada
sobre a governança social de alianças e redes de empresas. A
chave, aqui, está em construir debates de alto impacto acadêmico,
mas que efetivamente gerem avanços no conhecimento relevante.
Nesse sentido, McGahan (2007) traz uma série de
exemplos e sugestões sobre como aumentar a relevância
gerencial da pesquisa em Administração. Uma possibilidade
é buscar resultados surpreendentes que sejam, a princípio,
contraintuitivos, mas que possam abrir novas oportunidades
de refinamento teórico. Por exemplo, Khanna e Palepu (2000)
buscaram mostrar que a visão tradicional sobre os malefícios de
diversificação excessiva não é necessariamente verdadeira em
países carentes de boas instituições e mercados eficientes. Esse
ponto acabou gerando toda uma nova literatura sobre estratégia
em países emergentes e rapidamente passou a ser usado como
justificativa gerencial para a montagem (ou a permanência) de
grupos empresariais altamente diversificados, a despeito do que
ensina a maior parte dos livros-texto de estratégia.
 IMPACTO SOCIAL E NO DEBATE PÚBLICO
Há, adicionalmente, uma crescente preocupação de gerar
pesquisas que ajudem a entender problemas sociais complexos
e proponham possíveis encaminhamentos, senão soluções, para
esses problemas. Uma questão muito comumente examinada,
por exemplo, é como empresas podem atender objetivos
sociais e ambientais para além do exigido pelos seus acionistas.
Desenvolveu-se, em Administração, um volumoso corpo de
estudos teóricos e empíricos tentando examinar o papel de
ISSN 0034-7590
PERSPECTIVAS | PESQUISA EM ADMINISTRAÇÃO: EM BUSCA DE IMPACTO SOCIAL E OUTROS IMPACTOS
Sérgio Lazzarini
622 | © RAE | São Paulo | V. 57 | n. 6 | nov-dez 2017 | 620-625
diversos stakeholders na concepção das estratégias das empresas
e no seu desempenho final (Barnett, 2007; Freeman, 1984;
Garcia-Castro & Aguilera, 2015). Há, também, pesquisas mais
diretamente ligadas ao estudo de organizações com objetivos
eminentemente de impacto social ou ambiental, que podem ou
não buscar lucratividade ou sustentabilidade financeira como
objetivo conjunto (Battilana & Dorado, 2010; Comini, Barki,
& Aguiar, 2012). Mais ainda, pesquisadores têm examinado
as complexas interações entre atores públicos e estatais na
concepção e execução de projetos de interesse público (Cabral,
Lazzarini, & Azevedo, 2013; Kivleniece & Quelin, 2012; Mahoney,
McGahan, & Pitelis, 2009).
A despeito da importância dessas linhas de investigação,
uma pesquisa de alto impacto em Administração não precisa
necessariamente ser direcionada ao estudo de organizações
socialmente engajadas ou com interface pública. Seguindo a
minha discussão anterior, uma pergunta mais básica é se a
pesquisa está efetivamente contribuindo para gerar resultados
relevantes para o mundo gerencial. É curioso, por exemplo,
observar acadêmicos criticando determinados modelos de
amplo uso por gestores, mas que, na sua atividade de ensino,
acabam reforçando essas próprias teorias que tanto criticam!
Alguns professores de estratégia, por exemplo, denunciam as
limitações da abordagem estrutural de Porter (1980), porém usam
justamente o seu famoso “modelo cinco forças” para ensinar,
a alunos de graduação e MBA, formas de analisar indústrias e
setores. Se essa abordagem é falha, qual alternativa poderia ser
usada para capacitar e auxiliar profissionais de gestão?
Além de deixar mais claro como as pesquisas podem
informar ou até mesmo mudar práticas de gestão, há, também,
grande espaço para tornar o campo acadêmico de Administração
mais próximo do debate público. O caso que reportei na introdução
demonstra que há, aparentemente, uma lacuna entre o que é feito
em pesquisa em Administração e o que efetivamente é utilizado
ou considerado em discussões mais abertas com a sociedade.
Porém, aqui não parece haver um problema de repertório; há uma
grande diversidade de estudos em Administração tangenciando
inúmeros temas que usualmente aparecem no debate público.
Parece haver simplesmente falta de diálogo, de um maior esforço
para conectar os resultados das pesquisas e as discussões
aplicadas sobre temas relevantes e atuais.
Considere, por exemplo, um tópico discutido à exaustão
quando se fala sobre as causas do baixo crescimento da economia
brasileira: produtividade. Em tese, esse seria um tema sobre o
qual os pesquisadores em Administração deveriam ter muito a
dizer, especialmente porque, cada vez mais, se fala em como
aumentar a produtividade da economia por meio de melhores
práticas de gestão nos setores público ou privado. Curiosamente,
nesse campo, os economistas parecem ter tomado muito mais
espaço no debate que os acadêmicos de Administração, ainda
que esses últimos continuem criticando os primeiros por uma
visão excessivamente limitada e reducionista das organizações.
Os trabalhos de Bloom e Van Reenen (2010), por exemplo,
mostraram como práticas gerenciais se correlacionam com maior
produtividade das empresas em diversos países. Recentemente,
os resultados desses estudos foram tomados por acadêmicos de
estratégia propondo uma “visão baseada em práticas” (Bromiley
& Rau, 2014).
Por que, com raras exceções, os acadêmicos em
Administração não participam mais do debate sobre
produtividade? Em algumas conversas informais, já notei uma
certa aversão conceitual ao tema, que denotaria, para alguns,
uma visão antiga (taylorista) de gestão. Há, aqui, uma falha
de entendimento. Produtividade é apenas uma medida de
desempenho (o quanto determinados recursos geram de produto
ou riqueza, tudo o mais constante). Maior desempenho, por
sua vez, pode ser gerado por um conjunto de práticas distintas,
desde as tayoloristas clássicas (padronização de processos e
monitoramento) até aquelas tidas como mais modernas (decisão
em times, processos de construção coletiva de conhecimento,
gestão de redes organizacionais e assim por diante). Ao virar
as costas para o tema da produtividade, os acadêmicos de
Administração perdem a oportunidade de fazer contribuições
fundamentais ao conhecimento e à prática. Será que as medidas
de qualidade de gestão empregadas pelos economistas são
adequadas? Mesmo que essas práticas existam e aumentem a
produtividade, quais seriam os processos políticos, sociais ou
cognitivos que impediriam a sua adoção? Essas são perguntas
muito afeitas ao métier da pesquisa em Administração, mas que
raramente aparecem no debate público sobre como aumentar a
produtividade das empresas.
PUBLICAÇÃO INTERNACIONAL É
INCONSISTENTE COM IMPACTO LOCAL?
Um debate polêmico ligado aos dilemas de impacto da
pesquisa em Administração diz respeito ao direcionamento
das publicações. Respondendo a críticas de que poucos
pesquisadores em Administração buscam publicar em revistas
de elevado impacto no exterior, alguns acadêmicos argumentam
que pesquisas de maior relevância focalizam aspectos locais, o
que acaba reduzindo a sua aceitação em periódicos estrangeiros.
Um corolário dessa afirmação é que a disseminação de pesquisa
ISSN 0034-7590
PERSPECTIVAS | PESQUISA EM ADMINISTRAÇÃO: EM BUSCA DE IMPACTO SOCIAL E OUTROS IMPACTOS
Sérgio Lazzarini
623 | © RAE | São Paulo | V. 57 | n. 6 | nov-dez 2017 | 620-625
em contextos internacionais desvaloriza revistas construídas e
geridas pela comunidade local. Essas revistas, em tese, teriam
mais impacto no debate nacional do que veículos direcionados
a acadêmicos de outros países.
Há várias ressalvas a esse tipo de argumento. Primeiro,
será que a pesquisa que é publicada em periódicos locais é
efetivamente usada para a construção de conhecimento por
acadêmicos do país? O que se vê, em vários casos, são pesquisas
publicadas localmente, mas que buscam simplesmente aplicar
ou testar teorias internacionalmente desenvolvidas. Para que
as pesquisas locais se insiram nas redes globais de geração
de conhecimento, é importante que uma parte delas tente
mirar periódicos acessados e lidos por uma comunidade mais
ampla. Sabe-se, por exemplo, que a probabilidade de um artigo
ser usado e citado por outros acadêmicos aumenta quando é
publicado em periódicos de alto prestígio (Judge, Cable, Colbert,
& Rynes, 2007). Os pesquisadores em geral simplesmente
não conseguem acompanhar as dezenas de revistas locais ou
internacionais na sua área; sua atenção, portanto, naturalmente
recai sobre os periódicos mais importantes. Mais ainda, a própria
publicação da pesquisa em um veículo de destaque já serve
como um sinal (ainda que imperfeito) da sua qualidade, inovação
e rigor. Se os resultados de Baldridge et al. (2004), citados
antes, se mantiverem corretos, esse atributo também deverá se
correlacionar com a relevância percebida da pesquisa.
Segundo, não conheço evidência substantiva de que
a maior parte das publicações em periódicos locais seja
efetivamente utilizada por gestores ou ajude a iluminar questões
prementes no debate aplicado (Cabral & Lazzarini, 2011). Se
um artigo local simplesmente toma alguma abordagem teórica
já estabelecida e a ilustra com casos ou dados nacionais, nada
garantirá achados ou conclusões surpreendentes que chamem
a atenção de um público mais amplo. Além disso, não há nada
que impeça que uma pesquisa publicada internacionalmente seja
amplamente divulgada no contexto local e sirva como insumo
para importantes discussões gerenciais ou de políticas públicas.
Dito isso, é inegável que existem temas locais cuja
relevância pode ser pouco compreendida por editores e
revisores de artigos submetidos a publicação internacional.
Nesse sentido, as revistas nacionais e outros veículos (como
livros de maior divulgação publicados no país) sempre
continuarão sendo muito importantes. Mas isso não implica
que achados locais não possam receber destaque no circuito
global. A saída, nesse caso, é tentar encontrar problemas de
elevada relevância local cuja análise rigorosa consiga trazer
resultados surpreendentes e inovadores, resultados esses que,
quem sabe, possam até mesmo desafiar conceitos e teorias
tradicionalmente aceitas (Lazzarini, 2012). O exemplo anterior
das pesquisas identificando as implicações estratégicas para
empresas localizadas em países com instituições mais fracas
ilustra bem esse ponto. Com o avanço dos países emergentes e
os seus enormes desafios, até mesmo acadêmicos nos eixos de
pesquisa tradicionais passaram a se interessar por fenômenos
como a prevalência de grupos diversificados (Wan & Hoskisson,
2003), a presença do Estado como acionista (Cuervo-Cazurra,
Inkpen, Musacchio, & Ramaswamy, 2014), os efeitos da corrupção
(Jeong & Weiner, 2012), modelos de negócios desenhados para
atender consumidores na base da pirâmide (Prahalad, 2004),
entre vários outros.
EM BUSCA DE MAIS IMPACTO
São vários os possíveis caminhos para aumentar o impacto da
pesquisa em Administração. Sem a pretensão de ser exaustivo,
encerro o artigo com algumas sugestões, muitas delas resultantes
da minha própria experiência. Um ponto importante é sempre
tentar ancorar desenvolvimentos teóricos e testes empíricos a um
problema central, de elevada relevância no mundo gerencial ou no
campo das políticas públicas. Naturalmente, nós nos apegamos
a determinadas vertentes teóricas, com os seus pressupostos e
implicações. Nós nos autodenominamos especialistas em teoria
institucional, abordagem dos stakeholders ou visão baseada em
recursos. Mas essas teorias não existem em um vácuo; elas, em
geral, resultam da necessidade de entender melhor determinados
fenômenos. Por exemplo, no desenvolvimento da sua teoria dos
custos de transação, Williamson (1985) sempre deixou claro que
seu problema focal era explicar padrões de integração vertical das
firmas – um tema polêmico, dado que a presença das empresas
em múltiplos elos da cadeia produtiva poderia resultar em efeitos
anticompetitivos. Williamson buscou enfatizar possíveis ganhos
de eficiência trazidos pela integração vertical das firmas, em
função de diversos fatores e contingências (como o grau de
especificidade dos ativos). Apesar de ter foco econômico, a
pesquisa de Williamson teve alta repercussão em comunidades
de gestão interessadas em explicar e entender estratégias de
expansão corporativa nas cadeias produtivas.
Também pode ser muito importante manter contato
próximo com empresas, institutos e governos, trazendo questões
e problemas que possam representar oportunidades concretas
de pesquisa. Mas há, aqui, diversos cuidados a serem tomados.
Muitas organizações demandam consultoria para soluções que
resultam de aplicação direta de modelos ou metodologias já
conhecidas e estabelecidas, para as quais pouco esforço de
ISSN 0034-7590
PERSPECTIVAS | PESQUISA EM ADMINISTRAÇÃO: EM BUSCA DE IMPACTO SOCIAL E OUTROS IMPACTOS
Sérgio Lazzarini
624 | © RAE | São Paulo | V. 57 | n. 6 | nov-dez 2017 | 620-625
pesquisa é necessário. Porém, a academia realmente adicionará
valor quando auxiliar essas organizações a compreender ou
solucionar questões novas e cujo esforço analítico permitirá
melhor compreender determinados fenômenos aplicados. Em
se firmando parcerias com empresas e governos, é importante
também garantir que os pesquisadores tenham autorização para
publicar os resultados e achados dos projetos, e que eventuais
financiamentos externos desses projetos não comprometam a
imparcialidade e isenção dos pesquisadores.
Além disso, um exame atento a atividades e setores ainda
pouco estudados pode trazer oportunidades concretas de inovar,
com novas perspectivas e abordagens. Em 2010, Jay Barney e Anita
McGahan organizaram um workshop na Ohio State University
para discutir como pesquisadores de Administração poderiam
contribuir para debates de políticas públicas. Estava presente
no evento William Ouchi, um acadêmico de grande importância
no desenvolvimento de teorias organizacionais (Ouchi, 1980) e
que, mais recentemente, passou a atuar com gestão educacional
(Ouchi, 2008). Ouchi disse que, se fosse um aluno de doutorado
ou professor em início de carreira, construiria sua agenda de
pesquisa estudando organizações atípicas, como escolas, prisões
e hospitais. São esses contextos, segundo ele, que possivelmente
trarão novos elementos e achados para além do que já foi
estudado em setores tradicionais. Porém, até mesmo empresas
estabelecidas, em indústrias tradicionais, passam, cada vez mais,
a implantar ações para reduzir o impacto ambiental das suas
atividades, aumentar a diversidade de gênero e racial dos seus
executivos, lidar com políticas governamentais e distribuir valor
para parceiros ao longo das suas cadeias produtivas. São todos
temas inovadores, relevantes gerencialmente, de alto interesse
para o debate público e que, de quebra, podem representar
grandes oportunidades de construção e refinamento teórico.
Dessa forma, não precisamos nem devemos abandonar
ou desmerecer teorias, como dito na epígrafe deste artigo.
Mas, certamente, com um novo esforço investigativo, podemos
direcioná-las para temas prementes e pouco estudados. E, quem
sabe, mudar as coisas para melhor.
REFERÊNCIAS
Baldridge, D. C., Floyd, S. W., & Markóczy, L. (2004). Are managers from
Mars and academicians from Venus? Toward an understanding of
the relationship between academic quality and practical relevance.
Strategic Management Journal, 25(11), 1063-1074. doi:10.1002/smj.406
Barnett, M. L. (2007). Stakeholder influence capacity and the variability
of financial returns to corporate social responsibility. Academy of
Management Review, 32(3), 794-816.
Battilana, J., & Dorado, S. (2010). Building sustainable hybrid
organizations: The case of commercial microfinance organizations.
Academy of Management Journal, 53(6), 1419-1440. doi:10.5465/
amj.2010.57318391
Bloom, N., & Van Reenen, J. Van. (2010). Why do management
practices differ across firms and countries? The Journal of Economic
Perspectives, 24(1), 203-224.
Bromiley, P., & Rau, D. (2014). Towards a practice-based view of strategy.
Strategic Management Journal, 35(8), 1249-1256. doi:10.1002/
smj.2238
Cabral, S., & Lazzarini, S. G. (2011). Internacionalizar é preciso, produzir
por produzir não é preciso. Organizações & Sociedade, 18(58), 541-
542. doi:10.1590/S1984-92302011000300011
Cabral, S., Lazzarini, S. G., & Azevedo, P. F. (2013). Private entrepreneurs
in public services: A longitudinal examination of outsourcing and
statization of prisons. Strategic Entrepreneurship Journal, 7(1), 6-25.
doi:10.1002/sej.1149
Comini, G., Barki, E., & Aguiar, L. T. Dd. (2012). A three-pronged
approach to social business: A Brazilian multi-case analysis. RAUSPRevista de Administração da Universidade de São Paulo, 47(3), 385-
397. doi:10.1590/S0080-21072012000300004
Cuervo-Cazurra, A., Inkpen, A., Musacchio, A., & Ramaswamy, K.
(2014). Governments as owners: State-owned multinational
companies. Journal of International Business Studies, 45(8), 919-942.
doi:10.1057/jibs.2014.43
Finkelstein, S., & Hambrick, D. C. (1989). Chief executive compensation:
A study of the intersection of markets and political processes.
Strategic Management Journal, 10(2), 121-134. doi:10.1002/
smj.4250100203
Freeman, R. E. (1984). Strategic management: A stakeholder approach.
London, UK: Pitman.
Garcia-Castro, R., & Aguilera, R. V. (2015). Incremental value creation
and appropriation in a world with multiple stakeholders. Strategic
Management Journal, 36(1), 137-147. doi:10.1002/smj.2241
Granovetter, M. (1985). Economic action and social structure: The
problem of embeddedness. American Journal of Sociology, 91(3),
481-510.
Hoskisson, R. E., Castleton, M. W., & Withers, M. C. (2009).
Complementarity in monitoring and bonding: More intense
monitoring leads to higher executive compensation. The
Academy of Management Perspectives, 23(2), 57-74. doi:10.5465/
amp.2009.39985541
Jeong, Y., & Weiner, R. J. (2012). Who bribes? Evidence from the United
Nations’ oil-for-food program. Strategic Management Journal, 33(12),
1363-1383. doi:10.1002/smj.1986
Judge, T. A., Cable, D. M., Colbert, A. E., & Rynes, S. L. (2007). What
causes a management article to be cited: Article, author, or journal?
Academy of Management Journal, 50(3), 491-506. doi:10.5465/
amj.2007.25525577
Khanna, T., & Palepu, K. (2000). The future of business groups in
emerging markets: Long-run evidence from Chile. Academy of
Management Journal, 43(3), 268-285.
Kivleniece, I., & Quelin, B. V. (2012). Creating and capturing value
in public-private ties: A private actor’s perspective. Academy of
Management Review, 37(2), 272-299. doi:10.5465/amr.2011.0004
ISSN 0034-7590
PERSPECTIVAS | PESQUISA EM ADMINISTRAÇÃO: EM BUSCA DE IMPACTO SOCIAL E OUTROS IMPACTOS
Sérgio Lazzarini
625 | © RAE | São Paulo | V. 57 | n. 6 | nov-dez 2017 | 620-625
Latour, B. (1987). Science in action: How to follow scientists and
engineers through society. Cambridge, USA: Harvard University Press.
Lazzarini, S. G. (2012). Leveraging the competitive advantage
of Iberoamerican scholars. Management Research: Journal
of the Iberoamerican Academy of Management, 10(1), 64-73.
doi:10.1108/1536-541211228577
Mahoney, J. T., McGahan, A. M., & Pitelis, C. N. (2009). The
interdependence of private and public interests. Organization
Science, 20(6), 1034-1052. doi:10.1287/orsc.1090.0472
McGahan, A. M. (2007). Academic research that matters to managers:
On zebras, dogs, lemmings, hammers, and turnips. Academy of
Management Journal, 50(4), 748-753. doi:10.5465/amj.2007.26279166
Ouchi, W. G. (1980). Markets, bureaucracies, and clans. Administrative
Science Quarterly, 25(1), 129-141. doi:10.2307/2392231
Ouchi, W. G. (2008). Power to the principals: Decentralization in
three large school districts. Organization Science, 17(2), 298-307.
doi:10.1287/orsc.1050.0172
Porter, M. E. (1980). Competitive strategy. New York, USA: The Free Press.
Prahalad, C. K. (2004). The fortune at the bottom of the pyramid. Upper
Saddle River, USA: Pearson.
Wan, W. P., & Hoskisson, R. E. (2003). Home country environments,
corporate diversification strategies, and firm performance. Academy
of Management Journal, 46(1), 27-45.
Williamson, O. E. (1985). The economic institutions of capitalism. New
York, USA: The Free Press.
Zenger, T. R. (1992). Why do employers only reward extreme performance?
Examining the relationships among performance, pay, and turnover.
Administrative Science Quarterly, 37(2), 198-219. doi:10.2307/2393221
"""

In [95]:
teste = agente_classificador(query)



> Entering new AgentExecutor chain...

Invoking: `busca_gestao_publica` with `{'query': 'impacto da pesquisa em Administração no impacto social e no debate público'}`


Política com base em evidência: justificativa, uso e limites Toda política pública é motivada e voltada para a solução de um problema social. Invariavelmente, a busca por essa solução envolve modificar um resultado de interesse, seja impedindo a ocorrência de algo indesejável seja promovendo que algo desejável ocorra. Diante de uma variedade de problemas, com múltiplas estratégias de resolução e recursos escassos, todo gestor enfrenta escolhas difíceis. Para embasar e validar suas decisões, gestores de todo mundo vêm, cada vez mais, recorrendo ao uso de evidências, sejam elas pré-existentes, sejam elas geradas especificamente para apoiá-los em suas decisões. A tecnologia da informação e comunicação revolucionou as possibilidades de se fazer política pública com base em evidência, tanto por disponibilizar (e em boa med

In [96]:
print(teste)

```json
{
    "titulo": "PESQUISA EM ADMINISTRAÇÃO: EM BUSCA DE IMPACTO SOCIAL E OUTROS IMPACTOS",
    "classificacao": "Acadêmica",
    "metodologia": "Qualitativa",
    "area_avaliada": "Desenvolvimento Social",
    "etapa": "Justificativa",
    "ods": "9",
    "resumo_justificativa": "O artigo discute como a pesquisa em Administração pode aumentar seu impacto social e relevância no debate público, justificando a importância de conectar resultados de pesquisa com questões práticas e sociais."
}
```
